In [1]:
from __future__ import print_function
#%matplotlib inline
import argparse
import os
import random
import glob
import itertools
import pickle
from PIL import Image
import torch
import torchvision
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torchvision.utils import save_image
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
from sklearn.model_selection import train_test_split

from torchvision import datasets
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn.functional as F
from collections import OrderedDict


import networks
import data_loader

In [2]:
# pkl_file = open("numpyModel.pkl", "rb")
# data_ = pickle.load(pkl_file, encoding="latin1")
# dt = {}
# h = { "bn2.num_batches_tracked", "bn3.num_batches_tracked", "bn4.num_batches_tracked", "bn5.num_batches_tracked", "bn6.num_batches_tracked", "bn7.num_batches_tracked", "bn8.num_batches_tracked", "tr_bn1.num_batches_tracked", "tr_bn2.num_batches_tracked", "tr_bn3.num_batches_tracked", "tr_bn4.num_batches_tracked", "tr_bn5.num_batches_tracked", "tr_bn6.num_batches_tracked", "tr_bn7.num_batches_tracked"}
# for key, value in data_.items():
#     if key not in h:
#         dt[key] = torch.from_numpy(value) 
# torch.save(dt, "Model_Semantic_New.pth")

In [3]:
# Root directory for dataset
data_root = "/datasets/home/73/673/h6gupta/Project/Sematic_Segmentation/gt_for_cv/"
# Number of images in the directory
num_images = 800
# Batch size during training
batch_size = 32
# Spatial size of training images. All images will be resized to this size using a transformer.
image_size = 256
# Number of channels in the training images. For color images this is 3
nc = 3
# Size of feature maps in generator
ngf = 64
# Number of GPUs available. Use 0 for CPU mode.
ngpu = torch.cuda.device_count()
# Device to run on
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

def dice_coef(label, output):
    label = torch.round(label.view(-1))
    output = torch.round(output.view(-1))
    isct = torch.sum(torch.mul(label,output))
    return (2 *(isct) / (torch.sum(label) + torch.sum(output)))

In [4]:
data = data_loader.DataLoader(data_root, image_size, batch_size, train = False, folder_A = "images/", folder_B = "labels/")
GTA2CITY = networks.GeneratorUNet().to(device)
GTA2CITY.load_state_dict(torch.load('../../../../saved_models/dual_gans_semi/generator_b.pth_8_1462.pth'))
Semantic = networks.GeneratorUNet().to(device)
Semantic.load_state_dict(torch.load('Model_Semantic_Final.pth'))

In [5]:
iou = []
for i in range(num_images // batch_size):
    x,y = next(data.data_generator(0, train = False))
    img = Variable(x, requires_grad = False).to(device)
    lbl = Variable(y, requires_grad = False).to(device)
    
    img = (img - torch.min(img))/(torch.max(img - torch.min(img)))
    lbl = (lbl - torch.min(lbl))/(torch.max(lbl - torch.min(lbl)))
    
    cityscapes = GTA2CITY(img)
    cityscapes = (cityscapes - torch.min(cityscapes))/(torch.max(cityscapes - torch.min(cityscapes)))
    
    semantic_sample = Semantic(cityscapes)
    semantic_sample = (semantic_sample - torch.min(semantic_sample))/(torch.max(semantic_sample - torch.min(semantic_sample)))
    
    print (i, 'IOU: {}'.format(dice_coef(lbl, semantic_sample)))
    iou.append(dice_coef(lbl, semantic_sample).cpu().detach().numpy())
    for j in range(img.size()[0]):
        img_sample = torch.cat((img[j,:,:,:].data,lbl[j,:,:,:].data, cityscapes[j,:,:,:], semantic_sample[j,:,:,:]),-1)
        save_image(img_sample, '/datasets/home/73/673/h6gupta/Project/Sematic_Segmentation/MSELOSS/Final_Testing/outputs/%d.png' % ((batch_size*i) + j), nrow=2)
    

0 IOU: 0.7283569574356079
1 IOU: 0.7407826781272888
2 IOU: 0.7578699588775635
3 IOU: 0.7619194984436035
4 IOU: 0.7237129807472229
5 IOU: 0.7397133708000183
6 IOU: 0.7300137281417847
7 IOU: 0.7616379261016846
8 IOU: 0.7168512344360352
9 IOU: 0.7343743443489075
10 IOU: 0.775820791721344
11 IOU: 0.7310453653335571
12 IOU: 0.7310613393783569
13 IOU: 0.7451351284980774
14 IOU: 0.7333023548126221
15 IOU: 0.7511754631996155
16 IOU: 0.7480047941207886
17 IOU: 0.7672289609909058
18 IOU: 0.750859260559082
19 IOU: 0.7370699048042297
20 IOU: 0.7321837544441223
21 IOU: 0.7552876472473145
22 IOU: 0.7643251419067383
23 IOU: 0.7260404825210571
24 IOU: 0.7571142911911011


In [6]:
print ('Mean IOU: {}'.format(np.sum(np.asarray(iou))/(i+1)))

Mean IOU: 0.7240797424316406


In [8]:
# print(semantic_sample.size())
Loss = torch.nn.L1Loss()
loss = Loss(semantic_sample,lbl)
print(loss.item())

0.17722058296203613


32